In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
print(tf.test.gpu_device_name())
print(tf. __version__)

In [ ]:
"""
Mount Drive
"""
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
path = os.path.join('/content', 'drive', 'Sign language', 'DATASET')

In [ ]:
# Create a test and train directory
os.makedirs(os.path.join(path, 'train'))
os.makedirs(os.path.join(path, 'test'))

In [ ]:
# """
# Creating the test and train folders
# """

# Creating empty datafram for later splitting
df = pd.DataFrame(columns = ['loc', 'label'])

data = {'loc': '', 'label': ''}

# Load all images from 'data'
curr_path = os.path.join(path, 'DATASET')

# Create the train directory
for file in os.listdir(curr_path):   
    print(file)
    for item in os.listdir(os.path.join(curr_path, file)):
        data['loc'] = os.path.join(curr_path, file, item)
        data['label'] = file
        df = df.append(data, ignore_index = True)

df

In [ ]:
# # Splitting into two

train = df.sample(frac= 0.75, random_state=200) #random state is a seed value
test = df.drop(train.index)

In [ ]:
test.head()

In [ ]:
# # Saving the train dataset
train.to_csv(os.path.join(path, 'train_split.csv'))
test.to_csv(os.path.join(path, 'test_split.csv'))

In [ ]:
curr_path

In [ ]:
# """
# Moving images
# """

import cv2 as cv
import os
import matplotlib.pyplot as plt
import pandas as pd

train = pd.read_csv(os.path.join(path, 'train_split.csv'))

for _, row in train.iterrows():

    curr_path = os.path.join(path, 'train', row['label'])
  
    if not os.path.isdir(curr_path):
        os.makedirs(curr_path)
    
    image = cv.imread(row['loc'])

    # Use the cvtColor() function to grayscale the image
    rgb_image = cv.resize(cv.cvtColor(image, cv.COLOR_BGR2RGB), (128,128))

    
    loc_img = row['loc'].rfind('/')
    image_name = row['loc'][loc_img+1:-4] + '_02' + row['loc'][-4:] 
    img_path = os.path.join(curr_path, image_name)
    
    print(img_path)
    cv.imwrite(img_path, rgb_image)
  
test = pd.read_csv(os.path.join(path, 'test_split.csv'))

for _, row in test.iterrows():

    curr_path = os.path.join(path, 'test', row['label'])
        
    if not os.path.isdir(curr_path):
        os.makedirs(curr_path)
    
    image = cv.imread(row['loc'])
  
    # Use the cvtColor() function to grayscale the image
    rgb_image = cv.resize(cv.cvtColor(image, cv.COLOR_BGR2RGB), (128,128))
  
    loc_img = row['loc'].rfind('/')
    img_name = row['loc'][loc_img+1:-4] + '_02' + row['loc'][-4:] 
    img_path = os.path.join(curr_path, img_name)

    print(img_path)
    cv.imwrite(img_path, rgb_image)    

In [ ]:
"""
Model Architecture
"""

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sz = 128
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

In [ ]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


# Step 2 - Preparing the train/test data and training the model
classifier.summary()

In [ ]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

In [ ]:
"""
Image loading
"""
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

data_dir_tr = os.path.join(path, 'train')
data_dir_ts = os.path.join(path, 'test')

train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.5,
        shear_range=0.5,
        rotation_range=35,
        width_shift_range=0.1,
        height_shift_range=0.1,
        vertical_flip=True,)

test_datagen = ImageDataGenerator(
        rescale=1./255)



training_set = train_datagen.flow_from_directory(data_dir_tr,
                                                 target_size=(sz, sz),
                                                 batch_size=100,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(data_dir_ts,
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

In [ ]:
sz   = 128
batch_size = 100

with tf.device(tf.DeviceSpec(device_type="GPU", device_index='0')):
    classifier.fit_generator(
            training_set,
            steps_per_epoch= 120, # No of images in training set
            epochs=90,
            validation_data=test_set,
            validation_steps=400)# No of images in test set

In [ ]:
# Saving the model
model_json = classifier.to_json()

curr_path = os.path.join(path, 'models')

if not os.path.isdir(curr_path):
    os.makedirs(curr_path)
    

with open(os.path.join(curr_path, "custom_Model_json.json"), "w") as json_file:
    json_file.write(model_json)
print('Model json Saved')
classifier.save_weights(os.path.join(curr_path, 'custom_Model_weights.h5'))
print('Weights saved')
classifier.save(os.path.join(curr_path,'custom_Model.h5'))
print('Model saved')